## Lab 5: logistic regression. ROC. AUC. One-hot encoding

## 1
Consider the data from a health insurance company. Based on the characteristics of patients `Members.csv` (age, gender) and data on receiving medical care in the previous year `Claims_Y1.csv` (medical institution, doctor, type of problem, number of days of hospitalization, date, etc.), you need to predict the fact of hospitalization for at least 1 day in the next year `DaysInHospital_Y2.csv`.

In [ ]:
import pandas as pd

from sklearn import *

%matplotlib inline
import matplotlib.pyplot as plt

Read the data. Use `MemberID` as the value for parameter `index_col`. 

* data from the 'DaysInHospital_Y2.csv' table assign to the `days2` variable
* data from the 'Members.csv' table assign to the `m` variable
* data from the 'Claims_Y1.csv' table assign to the `claims` variable

In [ ]:
# place for code


## 2
To anonymize data, the organizer provided approximate information about patients, for example, the age column shows age groups: '0-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+'. Convert string attributes to quantitative ones and replace the missing values:

In [ ]:
i = pd.notnull(m.AgeAtFirstClaim)
m.loc[i,'AgeAtFirstClaim'] = m.loc[i,'AgeAtFirstClaim'].apply(lambda s: s.split('-')[0] if s!='80+' else '80')
m.loc[i,'AgeAtFirstClaim'] = m.loc[i,'AgeAtFirstClaim'].apply(lambda s: int(s))

m.AgeAtFirstClaim = m.AgeAtFirstClaim.fillna(value=-1)

m.Sex = m.Sex.fillna(value='N')

claims.CharlsonIndex = claims.CharlsonIndex.map({'0':0, '1-2':1, '3-4':3, '5+':5})
claims.LengthOfStay = claims.LengthOfStay.fillna(value=0)
claims.LengthOfStay = claims.LengthOfStay.map({0:0, '1 day':1, '2 days':2, '3 days':3, '4 days':4,\
    '5 days':5, '6 days':6, '1- 2 weeks':10, '2- 4 weeks':21, '4- 8 weeks':42, '26+ weeks':182})

## 3
Construct features using `claims` dta: 
* `f_Charlson` — the maximum index of Calson comorbidity for a patient (`CharlsonIndex` in a table `claims`)
* `f_LengthOfStay` — the total number of days of hospitalization last year (`LengthOfStay` in a table `claims`) 

*Functions that can be useful: `.groupby(['MemberID']), .max ().sum()`*

In [ ]:
f_Charlson = # place for code
f_LengthOfStay = # place for code

## 4

Let's create a matrix of features with columns: `f_Charlson`, `f_LengthOfStay`, patient's age, `ClaimsTruncated` (whether there were too many cases of medical care):

*Functions that can be useful: `.join()`*

In [1]:
data = days2
data = data.join(f_Charlson)

# place for code
data.head(5)

## 5
Create a function that will divide the sample into two parts `dataTrain` and `dataTest`, train logistic regression on `dataTrain`, apply it to `dataTest`, build an error curve and calculate the area under it:

In [ ]:
def calcAUC(data):
    dataTrain, dataTest = model_selection.train_test_split(data, test_size=0.5, random_state=1)
    model = linear_model.LogisticRegression()
    model.fit( dataTrain.loc[:, dataTrain.columns != 'DaysInHospital'], dataTrain.DaysInHospital )
    predictionProb = model.predict_proba( dataTest.loc[:, dataTest.columns != 'DaysInHospital'] )
    fpr, tpr, _ = metrics.roc_curve(dataTest['DaysInHospital'], predictionProb[:,1])
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2)
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.show()
    print( metrics.roc_auc_score(dataTest['DaysInHospital'], predictionProb[:,1]) )

## 6
Apply this function to `data`:

## 7
Logistic regression accepts only quantitative features as input.

Add the patient's gender to our data using one hot encoding:

*Functions that can be useful: `pd.get_dummies(m.Sex, prefix='pol')`*

## 8
Try applying one hot encoding to the existing features in `data2` or creating new features using the `claims` array.

Who built the feature matrix for which the logistic regression works with the best quality: +5 bonus points.